In [1]:
import numpy as np
import torch
from pytorch3d.io import load_obj

In [2]:
# m = load_obj("model_normalized.obj",load_textures=False)
# v_matrix = m[0]
# f_matrix = m[1].verts_idx

m = torch.load('mesh.pt')
v_matrix = m['verts']
f_matrix = m['faces']

In [3]:
def M_ijk_numpy(f_matrix,v_matrix,max_m):
    num_faces = len(f_matrix)
    
    # Extract coordinates
    A = np.zeros([num_faces,3])
    B = np.zeros([num_faces,3])
    C = np.zeros([num_faces,3])
    for face_num in range(num_faces):
        [A[face_num,:],B[face_num,:],C[face_num,:]] = v_matrix[f_matrix[face_num]]
    # Calculate Determinants
    dets = np.linalg.det([[A[i],B[i],C[i]] for i in range(len(f_matrix))])
    
    # Allocate Tensors
    M_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    C_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    D_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    S_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    
    # Calculate C Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        C_tensor[:,i,j,k] = (C[:,0]**i)*(C[:,1]**j)*(C[:,2]**k)*(np.math.factorial(i+j+k)/(np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k)))
                        
    # Calculate D Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # D_ijk=0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # D_ijk = 1
                            D_tensor[:,i,j,k] = 1
                        else:
                            D_tensor[:,i,j,k] = B[:,0]*D_tensor[:,i-1,j,k]+B[:,1]*D_tensor[:,i,j-1,k]+B[:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]
    
    
    # Calculate S Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # S_ijk = 0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # S_ijk = 1
                            S_tensor[:,i,j,k] = 1
                        else:
                            S_tensor[:,i,j,k] = A[:,0]*S_tensor[:,i-1,j,k]+A[:,1]*S_tensor[:,i,j-1,k]+A[:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]
        
    # Calculate M Tensor, parallellized over faces
    for i in range(max_m):
        for j in range(max_m):
            for k in range(max_m):
                if (i+j+k)<=max_m:
                    M_tensor[:,i,j,k] = ((np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))/np.math.factorial(i+j+k+3))*dets[:]*S_tensor[:,i,j,k]
    return np.sum(M_tensor,axis=0)


In [4]:
def moment_loss_numpy(f_matrix_1, v_matrix_1, f_matrix_2, v_matrix_2, max_m = 5):
    return np.linalg.norm(M_ijk_numpy(f_matrix_1, v_matrix_1,max_m)-M_ijk_numpy(f_matrix_2, v_matrix_2,max_m))

In [5]:
moment_loss_numpy(f_matrix,v_matrix,f_matrix,v_matrix)

0.0

In [6]:
M_ijk_numpy(f_matrix,v_matrix, 5)

array([[[ 5.98703027e-02, -4.12607817e-05,  5.80252691e-04,
         -1.17278317e-06,  1.07278499e-05],
        [-1.69435116e-03,  1.13828618e-06, -2.19107815e-05,
          4.32523016e-08, -4.77722066e-07],
        [ 3.33255694e-04, -2.33019491e-07,  3.21944244e-06,
         -6.53593615e-09,  0.00000000e+00],
        [-1.97563002e-05,  1.38016816e-08, -2.54804038e-07,
          0.00000000e+00,  0.00000000e+00],
        [ 3.71812533e-06, -2.59429009e-09,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00]],

       [[-1.61912308e-03,  1.64069324e-06, -1.35486198e-05,
          2.02691090e-08, -1.81595697e-07],
        [-1.18300484e-04,  8.71818110e-08, -7.49190105e-07,
          2.85166409e-09,  0.00000000e+00],
        [-6.39961301e-06,  9.69056539e-09, -3.87166542e-08,
          0.00000000e+00,  0.00000000e+00],
        [-1.07221057e-06,  3.57409392e-10,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [-3.90640180e-08,  0.00000000e+00,  0.00000000e+00,
  